# NexaAILLM

This example goes over how to use LangChain to interact with `NexaAI` models.

## Installation

In [ ]:
# install package
!pip install -U langchain-nexa-ai

## Environment Setup

You should get an api key from Nexa Hub and set it as an environment variable (NEXA_API_KEY)

In [ ]:
from langchain_nexa_ai import NexaAILLM

octopus_llm = NexaAILLM()

## Shopping Category

This example demonstrates how to use the NexaAILLM model to perform various shopping-related actions, including searching for products, viewing recommended products, and sorting products by price. The model is invoked with different prompts, and the results are processed to execute the corresponding actions.

### Define tool funtions

These functions will perform specific shopping-related actions based on Amazon website. You can customize these functions on your own. The following functions are examples demonstrating how to use the results from the NexaAILLM model.

In [ ]:
import urllib.parse
import webbrowser


# amazon search for shopping
def search_amazon_products(product_description: str):
    try:
        encoded_description = urllib.parse.quote(product_description)
        url = f"https://www.amazon.com/s?k={encoded_description}"
        webbrowser.open(url)
        print(f"Opened {url} in the Amazon.")
    except Exception as e:
        print("Failed to open URL:", str(e))


# amazon recommended products
def see_amazon_recommended_products(category_name: str):
    try:
        encoded_description = urllib.parse.quote(category_name)
        url = f"https://www.amazon.com/s?k={encoded_description}"
        webbrowser.open(url)
        print(f"Opened {url} in the Amazon.")
    except Exception as e:
        print("Failed to open URL:", str(e))


# We will use "kitchen appliance" to demo sort function in this case
def sort_amazon_products(criteria: str):
    try:
        criteria_map = {
            "relevance": "relevanceblender",
            "top sales": "exact-aware-popularity-rank",
            "most recent": "date-desc-rank",
            "price low to high": "price-asc-rank",
            "price high to low": "price-desc-rank"
        }
        sort_criteria = criteria_map.get(criteria, "relevanceblender")
        url = f"https://www.amazon.com/s?k=kitchen+appliance&s={sort_criteria}"
        webbrowser.open(url)
        print(f"Opened {url} in the Amazon.")
    except Exception as e:
        print("Failed to open URL:", str(e))

### Customize your prompt

You can customize the prompt to include any shopping-related queries.

In [ ]:
import json

# Search products
prompt = "Find laptops for gaming under $1000."
result = octopus_llm.invoke(prompt, category="shopping")
response = json.loads(result)
if "function_arguments" in response:
    arguments = response["function_arguments"]
    search_amazon_products(arguments[0])

# See recommended products
prompt = "Show recommended products in home appliances."
result = octopus_llm.invoke(prompt, category="shopping")
response = json.loads(result)
if "function_arguments" in response:
    arguments = response["function_arguments"]
    see_amazon_recommended_products(arguments[0])

# Sort products
prompt = "Show products sorted by low to high prices."
result = octopus_llm.invoke(prompt, category="shopping")
response = json.loads(result)
if "function_arguments" in response:
    arguments = response["function_arguments"]
    sort_amazon_products(arguments[0])

## Steaming Category

This example demonstrates how to use the NexaAILLM model to perform various streaming-related actions, including browsing content and searching for specific videos. The model is invoked with different prompts, and the results are processed to execute the corresponding actions.

### Define tool funtions

These functions will perform specific streaming-related actions based on YouTube website. You can customize these functions on your own. The following functions are examples demonstrating how to use the results from the NexaAILLM model.

In [ ]:
import webbrowser


# youtobe search content for streaming
def browse_youtube_content(categories: str):
    try:
        url = f"https://www.youtube.com/results?search_query={categories}"
        webbrowser.open(url)
        print(f"Opened {url} in the default browser.")
    except Exception as e:
        print("Failed to open URL:", str(e))


# youtobe search videos for streaming
def search_youtube_videos(query: str, content_creator: str):
    try:
        search_query = f"{query} {content_creator}".strip()
        url = f"https://www.youtube.com/results?search_query={search_query}"
        webbrowser.open(url)
        print(f"Opened {url} in the default browser.")
    except Exception as e:
        print("Failed to open URL:", str(e))

### Customize your prompt

You can customize the prompt to include any streaming-related queries.

In [ ]:
# Browse content
prompt = "Show me gaming channels."
result = octopus_llm.invoke(prompt, category="streaming")
response = json.loads(result)
if "function_arguments" in response:
    arguments = response["function_arguments"]
    browse_youtube_content(arguments[0]) if arguments else "all"

# Search videos
prompt = "List all music videos by Adele released in 2021."
result = octopus_llm.invoke(prompt, category="streaming")
response = json.loads(result)
if "function_arguments" in response:
    arguments = response["function_arguments"]
    query = arguments[0]
    content_creator = arguments[1] if len(arguments) > 1 and arguments[1] != "any" else ""
    search_youtube_videos(query, content_creator)

## Travel Category

This example demonstrates how to use the NexaAILLM model to perform various travel-related actions, including searching for accommodations, sorting hotels, and filtering properties by price and rating. The model is invoked with different prompts, and the results are processed to execute the corresponding actions.

### Define tool funtions

These functions will perform specific travel-related actions based on Expedia website. You can customize these functions on your own. The following functions are examples demonstrating how to use the results from the NexaAILLM model.

In [ ]:
import datetime


# Expedia search for travel accommodations
def search_expedia_accommodation(location: str, start_date: str, end_date: str):
    try:
        if location == "here":
            location = "New York"   # Replace with your default location

        today = datetime.datetime.now()
        start_date = (today.strftime('%Y-%m-%d') if start_date.lower() == "now" else
                      datetime.datetime.strptime(f"{start_date} {today.year}", '%B %d %Y').strftime('%Y-%m-%d'))

        end_date = ((today + datetime.timedelta(days=1)).strftime('%Y-%m-%d') if end_date.lower() == "tomorrow" else
                    datetime.datetime.strptime(f"{end_date} {today.year}", '%B %d %Y').strftime('%Y-%m-%d'))

        formatted_location = location.replace(" ", "%20")
        url = f"https://www.expedia.com/Hotel-Search?destination={formatted_location}&startDate={start_date}&endDate={end_date}"
        webbrowser.open(url)
        print(f"Opened {url} in the default browser.")
    except Exception as e:
        print("Failed to open URL:", str(e))


# Sorting Expedia accommodations
def sort_expedia_accommodations(criteria: str):
    try:
        criteria_map = {
            "price low to high": "PRICE_LOW_TO_HIGH",
            "price high to low": "PRICE_HIGH_TO_LOW",
            "distance from downtown": "DISTANCE",
            "guest rating": "REVIEW_RELEVANT",
            "star rating": "PROPERTY_CLASS"
        }
        sort_criteria = criteria_map.get(criteria, "RECOMMENDED")

        url = f"https://www.expedia.com/Hotel-Search?sort={sort_criteria}"
        webbrowser.open(url)
        print(f"Opened {url} in the default browser.")
    except Exception as e:
        print("Failed to open URL:", str(e))


# Filtering Expedia accommodations by price and rating
def filter_expedia_by_price_rating(price_max: float, rating_min: float):
    try:
        url = "https://www.expedia.com/Hotel-Search?"

        if price_max != -1:
            url += f"price=0&price={price_max}"
        if rating_min > 1:
            star_min = rating_min * 10
            stars = "&".join([f"star={star}" for star in range(star_min, 60, 10)])
            url += f"{stars}" if price_max == -1 else f"&{stars}"

        webbrowser.open(url)
        print(f"Opened {url} in the default browser.")
    except Exception as e:
        print("Failed to open URL:", str(e))

### Customize your prompt

You can customize the prompt to include any travel-related queries.

In [ ]:
# Search accommodation
prompt = "Find a Hotel to stay in Toyko from October 1 to October 10."
result = octopus_llm.invoke(prompt, category="travel")
response = json.loads(result)
if "function_arguments" in response:
    arguments = response["function_arguments"]
    location = arguments[0] if arguments else "here"
    start_date = arguments[1] if len(arguments) > 1 else "now"
    end_date = arguments[2] if len(arguments) > 2 else "tomorrow"
    search_expedia_accommodation(location, start_date, end_date)

# Sort accommodations
prompt = "Sort the hotels by their proximity to downtown."
result = octopus_llm.invoke(prompt, category="travel")
response = json.loads(result)
if "function_arguments" in response:
    arguments = response["function_arguments"]
    sort_expedia_accommodations(arguments[0])

# Filter by pricerating
prompt = "Show properties below $500 per night with a 4 stars rating or higher."
result = octopus_llm.invoke(prompt, category="travel")
response = json.loads(result)
if "function_arguments" in response:
    arguments = response["function_arguments"]
    price_max = int(arguments[0]) if arguments else -1
    rating_min = int(arguments[1]) if len(arguments) > 1 else 1
    filter_expedia_by_price_rating(price_max, rating_min)

## Using NexaAILLM for Streaming, Shopping, and Travel Scenarios

This example demonstrates how to use the NexaAILLM model to perform various actions across multiple categories, including streaming, shopping, and travel. The model is invoked with different prompts, and the results are processed to execute the corresponding actions.

In [ ]:
prompts=[
    "Show me Hawaii travel vlogs.",
    "Show recommended products for swimming.",
    "Find a Hotel to stay in Hawaii from August 1 to August 10.",
]

results = octopus_llm.generate(prompts, categories=["streaming", "shopping", "travel"])

for generation_list in results.generations:
    for generation in generation_list:
        response = json.loads(generation.text)
        if "function_name" in response and response["function_name"] and "function_arguments" in response:
            function_name = response["function_name"]
            arguments = response["function_arguments"]

            # shopping
            if function_name == "search_products":
                search_amazon_products(arguments[0])
            elif function_name == "see_recommended_products":
                see_amazon_recommended_products(arguments[0])
            elif function_name == "sort_products":
                sort_amazon_products(arguments[0])

            # streaming
            elif function_name == "browse_content":
                browse_youtube_content(arguments[0]) if arguments else 'all'
            elif function_name == "search_videos":
                query = arguments[0]
                content_creator = arguments[1] if len(arguments) > 1 and arguments[1] != "any" else ""
                search_youtube_videos(query, content_creator)

            # travel
            elif function_name == "search_accommodation":
                location = arguments[0] if arguments else "here"
                start_date = arguments[1] if len(arguments) > 1 else "now"
                end_date = arguments[2] if len(arguments) > 2 else "tomorrow"
                search_expedia_accommodation(location, start_date, end_date)
            elif function_name == "sort_accommodations":
                sort_expedia_accommodations(arguments[0])
            elif function_name == "filter_by_price_rating":
                price_max = int(arguments[0]) if arguments else -1
                rating_min = int(arguments[1]) if len(arguments) > 1 else 1
                filter_expedia_by_price_rating(price_max, rating_min)

            else:
                print("No valid function recognized.")
        else:
            print("No function name or arguments found in the model response.")

## Conclusion

This example demonstrates the versatile application of the NexaAILLM model across different categories, including streaming, shopping, and travel. You can further customize the agent to include more functionalities and integrate your application with other categories. By defining specific prompts and invoking the model, you can generate targeted responses that cater to various user needs.